In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Importing necessary Libraries
from sklearn.linear_model import Perceptron
from sklearn import metrics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

In [5]:
data = pd.read_csv("/content/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [6]:
data = data.drop(data.columns[9:-1], axis=1)

In [7]:
data.dtypes

,0
Time,int64
V1,float64
V2,float64
V3,float64
V4,float64
V5,float64
V6,float64
V7,float64
V8,float64
Class,float64


In [8]:
data.shape

(27819, 10)

In [9]:
data.isnull().sum()

,0
Time,0
V1,1
V2,1
V3,1
V4,1
V5,1
V6,1
V7,1
V8,1
Class,1


In [10]:
data.duplicated().sum()


np.int64(111)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27819 entries, 0 to 27818
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    27819 non-null  int64  
 1   V1      27818 non-null  float64
 2   V2      27818 non-null  float64
 3   V3      27818 non-null  float64
 4   V4      27818 non-null  float64
 5   V5      27818 non-null  float64
 6   V6      27818 non-null  float64
 7   V7      27818 non-null  float64
 8   V8      27818 non-null  float64
 9   Class   27818 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 2.1 MB


In [12]:
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(19473, 9)
(8346, 9)
(19473,)
(8346,)


In [14]:
#Importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [15]:
#To predict the accuracy
model_accuracies = {}
# Create a dictionary of models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}

In [18]:
# Iterate through the models and train/evaluate each one
for model_name, model in models.items():
  # Train the model
  # Reshape x_train and y_train to 2D using values.reshape(-1, 1) if necessary
  # Check if x_train is already 2D. If not, reshape
  if x_train.values.ndim == 1:  # Check if x_train is 1-dimensional
      x_train_reshaped = x_train.values.reshape(-1, 1)
  else:
      x_train_reshaped = x_train.values

  # Check if y_train is already 2D. If not, reshape
  if y_train.values.ndim == 1:  # Check if y_train is 1-dimensional
      y_train_reshaped = y_train.values.reshape(-1, 1)
  else:
      y_train_reshaped = y_train.values

  # Impute missing values using SimpleImputer for x_train
  from sklearn.impute import SimpleImputer
  imputer = SimpleImputer(strategy='mean')  # Replace 'mean' with other strategies if needed
  x_train_imputed = imputer.fit_transform(x_train_reshaped)

  #Handle Missing Values in y_train (Target Variable)
  # You can either remove rows with NaN or impute them.
  # Here, we remove rows with NaN in y_train
  from sklearn.impute import SimpleImputer #import SimpleImputer for handling missing values in target
  imputer_y = SimpleImputer(strategy='most_frequent') # Initialize SimpleImputer for y_train with the most frequent value. Other strategies can be used as well
  y_train_imputed = imputer_y.fit_transform(y_train_reshaped) # Fit and transform imputer_y to impute the missing values
  # If you chose to remove, comment out the above lines with SimpleImputer and uncomment the next line below.
  #x_train_imputed, y_train_imputed = x_train_imputed[~np.isnan(y_train_reshaped)], y_train_reshaped[~np.isnan(y_train_reshaped)] #remove rows with missing values in the y variable

  #Now fit the model with the imputed data
  model.fit(x_train_imputed, y_train_imputed.ravel()) # use ravel() to convert it back to a 1D array. This avoids a warning or potential errors.


  # Make predictions on the test set
  # Reshape x_test to 2D using values.reshape(-1, 1) if necessary
  if x_test.values.ndim == 1:  # Check if x_test is 1-dimensional
      x_test_reshaped = x_test.values.reshape(-1, 1)
  else:
      x_test_reshaped = x_test.values

  # Impute missing values in x_test as well
  x_test_imputed = imputer.transform(x_test_reshaped) #Use the same imputer fitted on training data


  y_pred = model.predict(x_test_imputed)
  accuracy = accuracy_score(y_test, y_pred)
  model_accuracies[model_name] = accuracy
  print(f"{model_name}: Accuracy = {accuracy}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Accuracy = 0.9968847352024922
Decision Tree: Accuracy = 0.9965252815720106
Random Forest: Accuracy = 0.9984423676012462
Support Vector Machine: Accuracy = 0.9959261921878745
K-Nearest Neighbors: Accuracy = 0.9960460100647016
Naive Bayes: Accuracy = 0.982266954229571


In [19]:
#max_accuracy
max_accuracy = 0
best_model = None
for model_name, accuracy in model_accuracies.items():
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    best_model = model_name

print("Best Model:", best_model)
print("Max Accuracy:", max_accuracy)

Best Model: Random Forest
Max Accuracy: 0.9984423676012462


In [21]:
#max_accuracy
max_accuracy = 0
best_model = None  # Initialize best_model to None
for model_name, accuracy in model_accuracies.items():
  if accuracy > max_accuracy:
    max_accuracy = accuracy
    best_model = model_name

# Check if best_model was found
if best_model is not None:
  print("Best Model:", best_model)
  print("Max Accuracy:", max_accuracy)
else:
  print("No best model found. All models have accuracy <= 0.")

#ipython-input-73-6a015efa903e
# Assuming 'best_model' is the best performing model
if best_model == "Random Forest":
  best_model = RandomForestClassifier()
elif best_model == "Logistic Regression":
  best_model = LogisticRegression()
elif best_model == "Decision Tree":
  best_model = DecisionTreeClassifier()
elif best_model == "Support Vector Machine":
  best_model = SVC()
elif best_model == "K-Nearest Neighbors":
  best_model = KNeighborsClassifier()
elif best_model == "Naive Bayes":
  best_model = GaussianNB()

# ... (previous code) ...

# Check if best_model is not None before fitting
if best_model is not None:
  # Train the best model on the entire dataset
  # Drop rows with NaN values in the target variable 'y'
  x_cleaned = x[~y.isnull()]  # Select rows where 'y' is not NaN
  y_cleaned = y[~y.isnull()]  # Select corresponding 'y' values
  best_model.fit(x_cleaned, y_cleaned)
else:
  print("Cannot train the model. 'best_model' is None.")

Best Model: Random Forest
Max Accuracy: 0.9984423676012462


In [22]:
#Gradio Interface
!pip install gradio

import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00


In [23]:
# prompt: invoke a predicted value for the above credit card fraud dettection code in gradio

def predict_fraud(V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V20, V21, V22, V23, V24, V25, V26, V27, V28, Amount):
    input_data = [[V1, V2, V3, V4, V5, V6, V7, V8, V9]]
    prediction = best_model.predict(input_data)[0]
    return prediction

iface = gr.Interface(
    fn=predict_fraud,
    inputs=[
        gr.Number(label="V1"), gr.Number(label="V2"), gr.Number(label="V3"), gr.Number(label="V4"), gr.Number(label="V5"),
        gr.Number(label="V6"), gr.Number(label="V7"), gr.Number(label="V8"), gr.Number(label="V9")],
    outputs=gr.Textbox(label="Fraud Prediction"),
    title="Credit Card Fraud Detection",
    description="Enter transaction details to predict fraud."
)

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 29 arguments for function <function predict_fraud at 0x79d8a862f6a0>, received 9.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 29 arguments for function <function predict_fraud at 0x79d8a862f6a0>, received 9.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b67f20f99039ee464b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [24]:
# Invoke predicted value in gradio

def predict_bmi(TransactionType, TransactionAmount, CustomerAge):
  new_data = pd.DataFrame({'TransactionType': [TransactionType], 'TransactionAmount': [TransactionAmount], 'CustomerAge': [CustomerAge]})
  predicted_index = best_model.predict(new_data)[0]



  if predicted_index == 0:
    return "Enter original values"
  elif predicted_index == 1:
    return "Underweight"
  elif predicted_index == 2:
    return "Normal"
  elif predicted_index == 3:
    return "Overweight"
  elif predicted_index == 4:
    return "Obesity"
  elif predicted_index == 5:
    return "Extreme Obesity"
  else:
    return "Unknown"



iface = gr.Interface(
    fn=predict_bmi,
    inputs=[
        gr.Dropdown(["Debit", "Credit"], label="TransactionType"),  # Use gr.Dropdown
        gr.Number(label="TransactionAmount"),  # Use gr.Number
        gr.Number(label="CustomerAge"),  # Use gr.Number
    ],
    outputs="text",  # Specify the output type as "text"
)

iface.launch(show_error=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c0859db4c1bcb32fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
